In [50]:
# Imports

import pandas as pd
import numpy as np
import re
from Scripts.normalize_original_data import getTrueLabelsDataframe
import Scripts.normalize_original_data as n
import Scripts.functions as fu
from pathlib import Path
import os


In [ ]:
df_ground_truth = getTrueLabelsDataframe()

## Loading Data to analyse


In [ ]:
bandit = pd.read_json('C:/Users/levim/projetos/results_mk2/formatted_bandit.json')
bandit['labels'] = bandit['labels'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

semgrep = pd.read_json('C:/Users/levim/projetos/results_mk2/formatted_semgrep.json')
semgrep['labels'] = semgrep['labels'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

ai_bandit = pd.read_json('C:/Users/levim/projetos/results_mk2/ai_results_bandit.json')


# ai_semgrep = pd.read_json('C:/Users/levim/projetos/results_mk2/ai_results_semgrep.json')




df_ground_truth = df_ground_truth.merge(bandit[['filename', 'labels']], on='filename', how='left').rename(columns={'labels':'bandit'})
df_ground_truth = df_ground_truth.merge(semgrep[['filename', 'labels']], on='filename', how='left').rename(columns={'labels':'semgrep'})

# df_ground_truth['bandit'] = df_ground_truth['bandit'].apply(
#     lambda x: ','.join(x) if isinstance(x, list) else x
# )


df_ground_truth


In [51]:
ai_bandit = pd.read_json('C:/Users/levim/projetos/results_mk2/ai_results_bandit.json')
ai_bandit['filename'] = ai_bandit['filename'].apply(lambda x: os.path.basename(x))

ai_bandit

,filename,ai_predictions
0,CWE-095_author_1.py,[]
1,XSS.py,[]
2,sqli.py,"[{'label': '42(CWE-94)', 'line of Code': 31}, ..."
3,CWE-918_codeql_1.py,"[{'label': 'CWE-400', 'line of Code': 13}]"
4,CWE-522_pearce_1.py,"[{'label': 'CWE-327', 'line of Code': 16}, {'l..."
...,...,...
72,CWE-454_author_1.py,"[{'label': '11(CWE-259)', 'line of Code': 11}]"
73,CWE-379_sonar_1.py,"[{'label': '7(CWE-377)', 'line of Code': 7}]"
74,simple_vulnerability.py,"[{'label': 'CWE-78', 'line of Code': 4}]"
75,CWE-521_sonar_2.py,"[{'label': '11(CWE-259)', 'line of Code': 14}]"
